In [1]:
#imports required for flask part
from flask import Flask, render_template,request
import os
import requests
from bs4 import BeautifulSoup
import nltk
import pandas as pd
from transformers import GPT2Tokenizer,GPT2LMHeadModel

app = Flask(__name__)
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model_gpt=GPT2LMHeadModel.from_pretrained('gpt2')

def get_wiki_content(url):
    req_obj = requests.get(url)
    text = req_obj.text
    soup = BeautifulSoup(text)
    all_paras = soup.find_all("p")
    wiki_text = ''
    for para in all_paras:
        wiki_text += para.text 
    return wiki_text


@app.route("/")
def msg():
    return render_template('index.html')

@app.route("/summarize",methods=['POST','GET'])
def getSummary():
    body=request.form.get('data')
    if len(body)==0:
        if request.method == 'POST':
            f = request.files['file']
            if(f.filename==""):
                url = request.form.get("url")
                url_content = get_wiki_content(url)
                inputs=tokenizer.batch_encode_plus([url_content],truncation=True, return_tensors='pt',max_length= 300)
                summary_ids=model_gpt.generate(inputs['input_ids'],early_stopping=True)
                result=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
                return render_template('summary_bert.html',result=result)
            else:
                temp1=request.form['size']
                temp1=int(temp1)
                f.save(f.filename)
                f.seek(0)
                content = f.read()
                content = str(content, 'utf-8')
                inputs=tokenizer.batch_encode_plus([content],truncation=True, return_tensors='pt',max_length= 300)
                summary_ids=model_gpt.generate(inputs['input_ids'],early_stopping=True)
                result=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
                return render_template('summary_bert.html', result=result)
    else:
        temp1=request.form['size']
        temp1=int(temp1)
        body=request.form.get('data')
        inputs=tokenizer.batch_encode_plus([body],truncation=True, return_tensors='pt',max_length= 300)
        summary_ids=model_gpt.generate(inputs['input_ids'],early_stopping=True)
        result=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
        return render_template('summary_bert.html',result=result)

if __name__ =="__main__":
    app.run()

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

C:\Users\vinee\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vinee\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jan/2023 15:54:50] "GET / HTTP/1.1" 200 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\vinee\anaconda3\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 300, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
127.0.0.1 - - [10/Jan/2023 15:55:02]